# Breakout - Policy Gradient Method
## Policy Gradients Vanilla type
* Borrowed heavily from structure of: [Vanilla Policy Gradient](https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb)
* In order to determine positive and negative rewards we must associate it with some direction of the pole

In [1]:
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, convolutional, pooling, Flatten, Dropout, Input
from keras.optimizers import Adam
from keras import backend as Bk
from scipy.misc import imread, imshow
import math
import numpy as np
import matplotlib.pyplot as plt
import gym
%matplotlib inline

Using TensorFlow backend.


In [2]:

env = gym.make('Breakout-v0')
print(env.observation_space)  # 210v x 160w x 3color Box (sparse tensor!!!)
print(env.action_space)       # 6x1 Discrete
s = env.reset()
print(s.shape)
gamma = 0.99
try: 
    xrange(1)
except:
    xrange = range

[2017-04-29 22:08:40,659] Making new env: Breakout-v0


Box(210, 160, 3)
Discrete(6)
(210, 160, 3)


In [3]:
class rl_agent():
    def __init__(self,  
                 environment, 
                 state_space_size, 
                 action_space_size,               
                 hidden_units = 200,
                 learning_rate = 0.1,             # Lambda or other for gradient descent
                 epsilon = 1e-8,                  # Error - error for types of gradient descent or random choice
                 random_choice_threshold = 0.07,
                 optimizer = Adam,
                 dropout_rate = 0.3,
                 load_file_weights = None,
                 verbose = 0,
                 frames = 3
                ):
        self.environment = environment
        self.hidden_units = hidden_units # hidden neurons
        self.state_space_size = state_space_size
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.optimizer = optimizer
        self.dropout_rate = dropout_rate
        self.load_file_weights = load_file_weights
        self.verbose = verbose
        self.frames = frames
        self.input_dim = 210*160
        self.prev_processed_state = None
        self.past_differences = []
        self.render = False
        self.random_choice_threshold = random_choice_threshold
        
    def define_model(self):
        # Keras vars
        self.model = Sequential()
        self.model.add(convolutional.Conv2D(filters=32, kernel_size = (3,3), strides=(1,1), padding='same',
            input_shape = (1, 210, 160), activation='relu'))
        self.model.add(pooling.MaxPooling2D(pool_size=(2,2), padding='same'))
        self.model.add(convolutional.Conv2D(filters=64, kernel_size = (5,5),
            strides=(1,1), padding='same', activation='relu'))
        self.model.add(pooling.MaxPooling2D(pool_size=(2,2), padding='same'))
        self.model.add(Flatten())
        self.model.add(Dense(self.hidden_units, activation='relu'))   # Karpathy suggested N=200 in the hidden layer
        self.model.add(Dropout(self.dropout_rate))      # But since we are doing dropout to avoid overfitting... increase
        self.model.add(Dense(6, activation='softmax'))  # Actions are a 6x1 vector
        optimizer = self.optimizer(lr=self.learning_rate)
        self.model.compile(
            optimizer = optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        try:
             self.model.load_weights('saved_weights.h5')
        except:
            print("Training model without old weights")
            
        # State space size will be a 210x160x1 dim
        self.state = Input(shape=self.state_space_size)
        self.state1 = Input(shape=self.state_space_size)
        #  self.current_action = tf.argmax(self.output_layer,1)
        self.reward_tensor = Input(shape=(1,), dtype='float32')
        self.action_tensor = Input(shape=(1,), dtype='int32')
        
        if self.verbose >= 1:
            print(self.model.summary())
        
    def training(self, max_games = 100000):
        mean_rewards = []
        self.state = self.environment.reset()
        chosen_vectors, rewards = [], []
        running_reward = 0
        reward_for_episode = 0
        i = 0
        while i < max_games:
            history_element, done = self.choose_action()
            chosen_vectors.append(history_element[4])
            reward_for_episode += float(history_element[3])
            rewards.append(history_element[3])
            if done:
                i += 1
                all_episode_features = np.vstack(agent.past_differences)
                all_episode_chosen_vectors = np.vstack(chosen_vectors)
                all_episode_rewards = np.vstack(rewards)
                discounted_episode_rewards = apply_gamma(all_episode_rewards)
                # Standard normal feature scaling 
                mean_discounted_rewards = np.mean(discounted_episode_rewards)
                discounted_episode_rewards -= mean_discounted_rewards
                discounted_episode_rewards /= np.std(discounted_episode_rewards)
                all_episode_chosen_vectors = np.multiply(discounted_episode_rewards.transpose(), all_episode_chosen_vectors.transpose()).transpose()
                self.model.fit(all_episode_features.reshape([all_episode_features.shape[0], 1, 210,160]), all_episode_chosen_vectors, epochs=1, verbose=2, shuffle=True)
                self.past_differences = []
                self.prev_processed_state = None
                chosen_vectors = []
                rewards = []
                reward_for_episode = 0
                self.state = self.environment.reset()
                if i%100 == 0:
                    self.saveweights()
                    mean_rewards.append(mean_discounted_rewards)
                    print("MEAN DISCOUNTED REWARDS:" + str(mean_discounted_rewards))
                    
    def savemodel(self):
        with open('saved_model.json') as modelfile:
            modelfile.write(self.model.to_json())
            
    def saveweights(self):
        self.model.save_weights('saved_weights.h5', overwrite=True)
        
    def choose_action(self):
        #Probabilistically pick an action given our network outputs.
        s = self.preprocess(self.state)
        #s = self.state
        self.prev_processed_state = self.prev_processed_state if self.prev_processed_state is not None else np.zeros((1,210, 160), dtype='float32')
        difference_processed = (s - self.prev_processed_state)
        self.prev_processed_state = s
        # difference_processed = difference_processed.flatten()
        self.past_differences.append(difference_processed.flatten())
        self.action_probabilities = self.model.predict(difference_processed.reshape([1,1,210,160])).flatten()
        e = np.random.normal()
        if (e < self.random_choice_threshold):
            action = np.random.choice(self.environment.action_space.n, 1)[0]
        else:
            action = np.random.choice(self.environment.action_space.n, 1,p=self.action_probabilities/np.sum(self.action_probabilities))[0]
        self.state, reward, done, info = self.environment.step(action)
        if self.render: self.environment.render()
        chosen_vector = np.zeros([self.environment.action_space.n])
        chosen_vector[action] = 1
        return [s,action, self.state, reward, chosen_vector], done
    
    def preprocess(self, state):
        # take a 210x160x3 image and convert to 210x160x1 as 210*160 length vector flattened F = float32 in dim
        # From https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.imread.html
        r, g, b = state[:, :, 0], state[:, :, 1], state[:, :, 2]
        F = r * 299.0/1000 + g * 587.0/1000 + b * 114.0/1000
        return F

In [4]:
def apply_gamma(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    r = r.flatten()
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


In [5]:
repeats = 999
max_games = 5000
agent = rl_agent(
    environment=env,
    state_space_size=(210,160,1),
    action_space_size=6,
    hidden_units=200,
    learning_rate=0.07,
    random_choice_threshold = 0.07,
    epsilon=1e-8, 
    optimizer=Adam,
    dropout_rate = 0.6, verbose=1)
agent.define_model()
agent.training()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 210, 32)        46112     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 105, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 105, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 53, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3392)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               678600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
__________

C:\Users\Carl-PC\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in true_divide


Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: 0.6007 - acc: 0.2521
Epoch 1/1
0s - loss: -4.8607e-01 - acc: 0.3478
Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: 0.4359 - acc: 0.2379
Epoch 1/1
0s - loss: -3.6525e-01 - acc: 0.2900
Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: 0.3628 - acc: 0.1193
Epoch 1/1
0s - loss: 0.1206 - acc: 0.2171
Epoch 1/1
0s - loss: -1.1619e-01 - acc: 0.3307
Epoch 1/1
0s - loss: -1.9748e-01 - acc: 0.3373
Epoch 1/1
0s - loss: -1.0862e-01 - acc: 0.3199
Epoch 1/1
0s - loss: 0.0935 - acc: 0.2638
Epoch 1/1
0s - loss: 0.0481 - acc: 0.2887
Epoch 1/1
0s - loss: 0.0530 - acc: 0.2992
Epoch 1/1
0s - loss: 0.1140 - acc: 0.2632
Epoch 1/1
0s - loss: 1.0158 - acc: 0.2500
Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: 0.5921 - acc: 0.2086
Epoch 1/1
0s - loss: nan - acc: 0.0000e+00
Epoch 1/1
0s - loss: 0.2689 - acc: 0.1000
Epoch 1/1
0s - loss: -8.4823e-01 - acc: 0.278

KeyboardInterrupt: 

In [ ]:
plt.hist(rewards)